In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [ ]:
% cd /content/gdrive/'My Drive'/'NLP'/'Final Project'/data
# !cd drive/NLP/'Final Project'

In [47]:
import math
import numpy as np
from collections import defaultdict

In [48]:
def get_word_and_tag(path):
    try:
        words = []
        tags = []
        word_tag = []
        f = open(path + '/train',encoding = 'utf-8')
        data = f.readlines()
        for x in data:
            if len(x) < 2:
#                 print(x)
                continue
            line = x.rstrip()
            temp = line.split(' ')
            word_tag.append(temp)
            words.append(temp[0])
            tags.append(temp[1])
        return words, tags, word_tag
    finally:
        f.close()
        
def get_dictionaries(word_tag):
    label_count = dict()
    for i in word_tag:
        label_count[i[1]] = label_count.get(i[1], 0) + 1
        
    word_count = dict()
    for i in word_tag:
        word_count[i[0]] = word_count.get(i[0], 0) + 1
    
    y_to_x = dict()
    for i in word_tag:
        y_to_x[i[1]+'+'+i[0]] = y_to_x.get(i[1]+'+'+i[0], 0) + 1
          
    return label_count, word_count, y_to_x

In [49]:
dataset = 'partial'
data_word, data_tags, data_word_tag = get_word_and_tag(dataset)
data_label_count, data_word_count, data_y_to_x = get_dictionaries(data_word_tag)

### Emissions


In [50]:
def get_emission(x, y, xy_dict, label_dict):
    if y+'+'+x not in xy_dict:
        return 0.0
    count_y_to_x = xy_dict[y+'+'+x]
    count_y = label_dict[y]
    return count_y_to_x / count_y

def get_emission_dict(word_and_tag,y_to_x,y_counts):
  ret = dict()
  for i in word_and_tag:
    ret['emission:'+i[1]+'+'+i[0]] = math.log(get_emission(i[0],i[1],y_to_x,y_counts))
  return ret

In [51]:
emission_dictionary = get_emission_dict(data_word_tag, data_y_to_x, data_label_count)
emission_dictionary

{'emission:O+The': -4.333515843240958,
 'emission:O+official': -7.166729187297174,
 'emission:O+cause': -8.776167099731275,
 'emission:O+of': -3.518671727703493,
 'emission:O+death': -7.166729187297174,
 'emission:O+has': -4.649032714686183,
 'emission:O+not': -5.97280671882474,
 'emission:O+been': -5.755742213586912,
 'emission:O+officially': -8.370701991623111,
 'emission:O+determined': -8.370701991623111,
 'emission:O+,': -3.3359162372945717,
 'emission:O+but': -6.211217742269738,
 'emission:O+investigators': -9.46931428029122,
 'emission:O+believe': -9.46931428029122,
 'emission:O+the': -2.8532490951584024,
 'emission:O+36-year-old': -9.46931428029122,
 'emission:O+writer': -9.46931428029122,
 'emission:O+died': -7.389872738611384,
 'emission:O+from': -5.192648161275165,
 'emission:O+a': -3.7522865788849984,
 'emission:O+self-inflicted': -9.46931428029122,
 'emission:O+gunshot': -9.46931428029122,
 'emission:O+wound': -9.46931428029122,
 'emission:O+.': -2.922528869530696,
 'emissi

### Transitions

In [52]:
def get_y_to_y(path):   
    try:
        f = open(path, encoding = 'utf-8')
        try_y = f.readlines()
    finally:
        f.close()
    sentences = []
    sentences.append('START')
    for line in try_y:
        x = line.split(' ')
    #     print(x)
        if len(x) <2:
            # linebreak
            sentences.append('STOP')
            sentences.append('START')
        else:
            sentences.append(x)
    states = []
    
    for term in range(len(sentences)-1): ## removes the last start
        if len(sentences[term])==2:
            states.append(sentences[term][1])
        else:
            states.append(sentences[term])
    for state in range(len(states)):
        states[state] = states[state].rstrip()
    return states

def get_y_to_y_count(states):
    y_to_y = dict()
    for i in range(len(states)-1):
        if(states[i+1]=='START'):
          continue
        y_to_y['transition:'+states[i]+ '+' +states[i+1]] = y_to_y.get('transition:'+states[i]+ '+'+states[i+1], 0) + 1 
        
    label_count_with_ss = dict()
    for i in range(len(states)):
        label_count_with_ss[states[i]] = label_count_with_ss.get(states[i], 0) + 1 
    return y_to_y , label_count_with_ss



In [53]:
data_states = get_y_to_y(dataset + "/train")
y_to_y_counts, labels_with_start_stop = get_y_to_y_count(data_states)
data_states

['START',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'B-per',
 'I-per',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'B-tim',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-org',
 'O',
 'B-per',
 'I-per',
 'I-per',
 'I-per',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-tim',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'B-gpe',
 'O',
 'O',
 'O',
 'B-tim',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'STOP',
 'START',
 'B-gpe'

In [54]:
def get_transmission(u, v, yy_dict, label_dict):
    # u is i , v is i+1
    if 'transition:'+u+'+'+v not in yy_dict:
        return 0.0
    count_u_to_v = yy_dict['transition:'+u+'+'+v]
    count_y = label_dict[u]
    result = count_u_to_v / count_y
    return result

def get_transmission_dict(emission_dict, y_to_y, y_count, states):
    for i in range(len(states)-1):
        if (states[i+1]=='START'):
            continue
        emission_dict['transition:'+states[i]+'+'+states[i+1]] = math.log(get_transmission(states[i],states[i+1],y_to_y,y_count))
    return emission_dict

In [55]:
resulting_dict = get_transmission_dict(emission_dictionary, y_to_y_counts, labels_with_start_stop, data_states)
resulting_dict

{'emission:O+The': -4.333515843240958,
 'emission:O+official': -7.166729187297174,
 'emission:O+cause': -8.776167099731275,
 'emission:O+of': -3.518671727703493,
 'emission:O+death': -7.166729187297174,
 'emission:O+has': -4.649032714686183,
 'emission:O+not': -5.97280671882474,
 'emission:O+been': -5.755742213586912,
 'emission:O+officially': -8.370701991623111,
 'emission:O+determined': -8.370701991623111,
 'emission:O+,': -3.3359162372945717,
 'emission:O+but': -6.211217742269738,
 'emission:O+investigators': -9.46931428029122,
 'emission:O+believe': -9.46931428029122,
 'emission:O+the': -2.8532490951584024,
 'emission:O+36-year-old': -9.46931428029122,
 'emission:O+writer': -9.46931428029122,
 'emission:O+died': -7.389872738611384,
 'emission:O+from': -5.192648161275165,
 'emission:O+a': -3.7522865788849984,
 'emission:O+self-inflicted': -9.46931428029122,
 'emission:O+gunshot': -9.46931428029122,
 'emission:O+wound': -9.46931428029122,
 'emission:O+.': -2.922528869530696,
 'emissi

# Part 2
#### (i)

In [56]:
def compute_crf_score(x, y, feature_dict):
    feature_count = {}
    for i in range(len(x)):
        formatted_word = x[i]
        emission_key = "emission:"+str(y[i])+"+"+str(formatted_word)
        feature_count[emission_key] = feature_count.get(emission_key, 0) + 1
    updated_y = ["START"]
    updated_y.extend(y)
    updated_y.append("STOP")
    for i in range(0, len(x)+1):
        prev_y = updated_y[i]
        y_i = updated_y[i+1]
        transition_key = "transition:"+str(prev_y)+"+"+str(y_i)
        feature_count[transition_key] = feature_count.get(transition_key, 0) + 1
    score = 0
    for k, v in feature_count.items():
        weight = feature_dict[k]
        score += weight * v
    return score

# # Test function
# x = "unidentifed defense probe generally confirmed allegations".split()
# y = "O O O O O O".split()
# compute_crf_score(x, y, resulting_dict)

#### (ii)

In [57]:
def viterbi_decode(x, states, feature_dict, default_index=0):
    n = len(x)
    d = len(states)
    scores = np.full((n, d), -np.inf)
    bp = np.full((n, d), default_index, dtype=np.int)

    for i in range(len(states)):
        transition_key = "transition:START"+states[i]
        emission_key = "emission:"+states[i]+"+"+x[0]
        transmission_score = feature_dict.get(transition_key, -9999999)
        emission_score = feature_dict.get(emission_key, -9999999)
        scores[0, i] = transmission_score + emission_score
    
    for i in range(1, n):
        for k in range(len(states)):
            #prev_y uses k, currenty uses j
            for j in range(len(states)):
                transition_key = "transition:"+states[k]+"+"+states[j]
                emission_key = "emission:"+states[j]+"+"+x[i]
                transition_score = feature_dict.get(transition_key, -9999999)
                emission_score = feature_dict.get(emission_key, -9999999)
                overall_score = emission_score + transition_score + scores[i-1, k]
                if overall_score > scores[i, j]:
                   scores[i, j] = overall_score
                   bp[i,j] = k
    
    highest_score = -9999999
    highest_bp = default_index
    for i in range(len(states)):
        transition_key = "transition:"+states[i]+"+STOP"
        transition_score = feature_dict.get(transition_key, -9999999)
        overall_score = transition_score + scores[n-1, i]
        
        if overall_score > highest_score:
            highest_score = overall_score
            highest_bp = i
    
    # Follow backpointers to get output sequence
    result = [states[highest_bp]]
    prev_bp = highest_bp
    for i in range(n-1, 0, -1):
        prev_bp = bp[i, prev_bp]
        output = states[prev_bp]
        # Prepend result to output list
        result = [output] + result
    
    return result

states = list(set(data_tags))
viterbi_decode("unidentifed defense probe generally confirmed allegations".split(), states, resulting_dict) # Should be similar or equal to y

['O', 'O', 'O', 'O', 'O', 'B-gpe']

In [58]:
list(set(data_tags))


['B-gpe',
 'I-geo',
 'B-art',
 'B-geo',
 'O',
 'I-org',
 'B-eve',
 'B-tim',
 'B-org',
 'I-per',
 'B-nat',
 'I-art',
 'I-tim',
 'I-eve',
 'B-per',
 'I-gpe']

In [59]:
def decode(path, states, feature_dict,output_filename):
    default_index = states.index('O')
    sentences = list()

    with open(path) as f:
        lines = f.readlines()
        sentence = list()
        for line in lines:
            formatted_line = line.strip()   
                 
            ## end of the sentence, so we continue to iterate
            if(len(formatted_line) ==0):
                sentences.append(sentence)
                sentence = []
                continue    
            sentence.append(formatted_line)

    with open(output_filename, "w") as wf:
        for sentence in sentences:
            pred_sentence = viterbi_decode(sentence, states, feature_dict, default_index)        
            for i in range(len(sentence)):
                
                ## writing the tags based on decoding
                wf.write(sentence[i] + " " + pred_sentence[i] + "\n")   
                
            ## EOL character
            wf.write("\n")

decode(dataset + "/dev.in", states, resulting_dict, dataset+'/dev.p2.out')

In [60]:

from conlleval2 import evaluate, evaluate_conll_file
def eval(pred,gold,out):
    f_pred = open(pred,encoding = 'utf-8')
    f_gold = open(gold,encoding = 'utf-8')
    data_pred = f_pred.readlines()
    data_gold = f_gold.readlines()
    gold_tags = list()
    pred_tags = list()
#     print(data_pred[0].split(' ')[0])
    
    with open(out, "w") as wf:
        for sentence in range(len(data_pred)):
        
            words_pred = data_pred[sentence].strip().split(' ')
#             print(words_pred)
        
            words_gold = data_gold[sentence].strip().split(' ')  
            if len(words_gold)==1:
#                 wf.write('\n')
                continue
            # Write original word and predicted tags
#             print(words_pred[0] + " " + words_pred[1] + " " + words_gold[1])
            gold_tags.append(words_gold[1])
            pred_tags.append(words_pred[1])
            wf.write(words_pred[0] + " " + words_pred[1] + " " + words_gold[1] +"\n")
            # End of sentence, write newline
    return gold_tags,pred_tags


g_tags, p_tags = eval(dataset+'/dev.p2.out',dataset+'/dev.out',dataset+'/eval.out')
print(evaluate(g_tags,p_tags,verbose=True))

processed 2097 tokens with 236 phrases; found: 203 phrases; correct: 135.
accuracy:  55.59%; (non-O)
accuracy:  91.03%; precision:  66.50%; recall:  57.20%; FB1:  61.50
              art: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
              eve: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              geo: precision:  81.43%; recall:  67.06%; FB1:  73.55  70
              gpe: precision:  88.89%; recall:  64.00%; FB1:  74.42  18
              nat: precision:   0.00%; recall:   0.00%; FB1:   0.00  13
              org: precision:  43.75%; recall:  40.00%; FB1:  41.79  32
              per: precision:  75.00%; recall:  46.88%; FB1:  57.69  20
              tim: precision:  70.21%; recall:  62.26%; FB1:  66.00  47
(66.50246305418719, 57.20338983050848, 61.503416856492024)


# Part 3
#### (i)

In [61]:
# This gives a damn huge loss, which i think is wrong
# YQ
# def compute_forward_score(x, feature_dict, states):
#     forward_scores = np.zeros((len(x), len(states)))
#     threshold = 999
    
#     # Start forward pass
#     # Compute START transition scores
#     for i in range(len(states)):
#         transition_key = "transition:START"+states[i]
#         transition_score = feature_dict.get(transition_key, -9999999)
#         emission_key = "emission:"+states[i]+"+"+x[0]
#         emission_score = feature_dict.get(emission_key, -9999999)
#         # Sum exponentials
#         forward_scores[0, i] = transition_score + emission_score
    
#     # Recursively compute best scores based on transmission and emission scores at each node
#     for i in range(1, len(x)):
#         for k in range(len(states)):
#             temp_score = 0
#             for j in range(len(states)):
#                 transition_key = "transition:"+states[j]+states[k]
#                 transition_score = feature_dict.get(transition_key, -9999999)
#                 emission_key = "emission:"+states[k]+x[i]
#                 emission_score = feature_dict.get(emission_key, -9999999)
#                 score = emission_score + transition_score + forward_scores[i-1,j]
#                 if score > threshold:
#                     score = threshold
#                 temp_score += np.exp(score)
                
#             # Add to forward scores array
#             forward_scores[i, k] = np.log(temp_score) if temp_score else -9999999

#     # Compute for STOP
#     forward_prob = 0
#     for j in range(len(states)):
#         transition_key = "transition:"+states[j]+"STOP"
#         transition_score = feature_dict.get(transition_key, -9999999)
#         # Sum exponentials
#         score = transition_score + forward_scores[len(x)-1,j]
#         if score > threshold:
#             score = threshold
#         overall_score = np.exp(score)
#         forward_prob += overall_score
#     # End forward pass
#     if forward_prob > 0:
#         alpha = np.log(forward_prob)
#     else:
#         alpha = threshold
        
#     return forward_scores, alpha

#YQ
def compute_forward_score(x, feature_dict, states):
    n = len(x) # Number of words
    d = len(states) # Number of states
    forward_scores = np.zeros((n, d))
    
    # Start forward pass
    # Compute START transition scores
    for i in range(len(states)):
        transition_key = "transition:START+"+states[i]
        transition_score = feature_dict.get(transition_key, -9999999)
        emission_key = "emission:"+states[i]+"+"+x[0]
        emission_score = feature_dict.get(emission_key, -9999999)
        # Sum exponentials
        forward_scores[0, i] = transition_score + emission_score
    
    # Recursively compute best scores based on transmission and emission scores at each node
    for i in range(1, n):
        for k, current_y in enumerate(states):
            temp_score = 0
            for j, prev_y in enumerate(states):
                transition_key = f"transition:{prev_y}+{current_y}"
                emission_key = f"emission:{current_y}+{x[i]}"
                
                transition_score = feature_dict.get(transition_key, -99999999)
                emission_score = feature_dict.get(emission_key, -99999999)
                
                # Sum exponentials
                temp_score += np.exp(min(emission_score + transition_score + forward_scores[i-1, j], 700))

            # Add to forward scores array
            forward_scores[i, k] = np.log(temp_score) if temp_score else -99999999

    # Compute for STOP
    forward_prob = 0
    for j, prev_y in enumerate(states):
        transition_key = f"transition:{prev_y}+STOP"
        transition_score = feature_dict.get(transition_key, -99999999)
        # Sum exponentials
        overall_score = np.exp(min(transition_score + forward_scores[n-1, j], 700))
        forward_prob += overall_score
    # End forward pass
    
    alpha = np.log(forward_prob) if forward_prob else -700
    return forward_scores, alpha


# -45.37727
_, a = compute_forward_score("Great food with an awesome atmosphere !".split(), resulting_dict, states)
a
compute_forward_score(['hola', 'buenas', 'agradezco', 'a', 'todas', 'las', 'personas', 'que', 'hayan', 'puesto', 'estos', 'comentarios', 'porque', 'esto', 'la', 'verdad', 'te', 'deja', 'saber', 'lo', 'bueno', 'y', 'lo', 'malo', 'que', 'tiene', 'el', 'restaurante', 'pero', 'yo', 'recomiendo', 'a', 'todas', 'las', 'personas', 'este', 'restaurante', 'porque', 'muy', 'pocos', 'restaurantes', 'en', 'zaragoza', 'tienen', 'pruductos', 'que', 'tiene', 'el', 'puerto', 'como', 'por', 'ejemplo', 'el', 'pescado', 'es', 'de', 'muy', 'alta', 'calidad', 'y', 'la', 'carne', 'es', 'muy', 'buena', 'que', 'es', 'de', 'denominacion', 'de', 'origen', 'y', 'hay', 'platos', 'muy', 'sabrosos', 'y', 'lo', 'bueno', 'que', 'tiene', 'el', 'restaurante', 'es', 'que', 'prepara', 'sus', 'deleciosos', 'platos', 'al', 'momento', 'nunca', 'se', 'prepara', 'la', 'comida', 'y', 'luego', 'se', 'calienta', 'y', 'con', 'un', 'tiempo', 'justo', 'para', 'el', 'cliente', 'y', 'con', 'mucha', 'limpieza', 'en', 'cocina', 'y', 'con', 'mucho', 'cuidado', 'con', 'el', 'pescado', 'y', 'demas', 'pruductos', 'y', 'con', 'excelentes', 'postres', 'caseros', 'como', 'la', 'quesa', 'pasiega', ',', 'pudin', 'de', 'frutas', ',', 'bolas', 'de', 'melon', 'maceradas', 'en', 'vino', 'de', 'pedro', 'xmenez', ',', 'tarta', 'imperial', 'de', 'almendras', 'y', 'chocolate', ',', 'etc', '.'], resulting_dict, states)

(array([[-10000001.59983662, -19999998.        , -19999998.        , ...,
         -19999998.        , -10000001.39219725, -19999998.        ],
        [-99999999.        , -99999999.        , -99999999.        , ...,
         -99999999.        , -99999999.        , -99999999.        ],
        [-99999999.        , -99999999.        , -99999999.        , ...,
         -99999999.        , -99999999.        , -99999999.        ],
        ...,
        [-99999999.        , -99999999.        , -99999999.        , ...,
         -99999999.        , -99999999.        , -99999999.        ],
        [-99999999.        , -99999999.        , -99999999.        , ...,
         -99999999.        , -99999999.        , -99999999.        ],
        [-99999999.        , -99999999.        , -99999999.        , ...,
         -99999999.        , -99999999.        , -99999999.        ]]),
 -700)

In [62]:
#ANDRE
def get_dataset(path):
    input_sequences = []
    input_labels = []
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        labels = []
        for line in lines:
            formatted_line = line.strip()
            if len(formatted_line) > 0:
                split_data = formatted_line.split(" ")
                x, y = split_data[0], split_data[1]
                sentence.append(x)
                labels.append(y)
            else:
                input_sequences.append(sentence)
                input_labels.append(labels)
                sentence = []
                labels = []
    return input_sequences, input_labels

def compute_crf_loss(input_sequences, input_labels, feature_dict, states, nabla=0, regularization=False):
    loss = 0
    for i in range(len(input_sequences)):
        first_term = compute_crf_score(input_sequences[i], input_labels[i], feature_dict)
        _, forward_score = compute_forward_score(input_sequences[i], feature_dict, states)
        loss += (first_term - forward_score) * -1
        
    if regularization:
        reg_loss = 0
        for feature_key in feature_dict:
            reg_loss += feature_dict[feature_key]**2
        reg_loss = nabla * reg_loss
        loss += reg_loss
    return loss

In [63]:
states = list(set(data_tags))
train_inputs, train_labels = get_dataset(dataset + "/train")
compute_crf_loss(train_inputs, train_labels, resulting_dict, states)

465.4578456572404

In [64]:
train_inputs

[['The',
  'official',
  'cause',
  'of',
  'death',
  'has',
  'not',
  'been',
  'officially',
  'determined',
  ',',
  'but',
  'investigators',
  'believe',
  'the',
  '36-year-old',
  'writer',
  'died',
  'from',
  'a',
  'self-inflicted',
  'gunshot',
  'wound',
  '.'],
 ['Mr.',
  'Omi',
  'called',
  'for',
  'stronger',
  'political',
  'will',
  'by',
  'Asian',
  'governments',
  'to',
  'stop',
  'the',
  'spread',
  'of',
  'the',
  'disease',
  '.'],
 ['This',
  'is',
  'the',
  'second',
  'U.N.-Congolese',
  'offensive',
  'against',
  'militias',
  'in',
  'the',
  'region',
  'since',
  'the',
  'DRC',
  "'s",
  'constitutional',
  'referendum',
  'a',
  'week',
  'ago',
  '.'],
 ['Wednesday',
  ',',
  'officials',
  'at',
  'the',
  'meeting',
  ',',
  'including',
  'Sudan',
  "'s",
  'President',
  'Omar',
  'Hassan',
  'al-Bashir',
  ',',
  'asked',
  'mediators',
  'to',
  'prepare',
  'for',
  'a',
  'new',
  'round',
  'of',
  'talks',
  'later',
  'this',
  'm

#### (ii)

In [65]:
#MIGUEL
def compute_backward_score(x, feature_dict, states):
    
    backward_scores = np.zeros((len(x), len(states)))
#     reversed_x = x[::-1]
    threshold = 700
    
    for i in range(len(states)):
        transition_key = "transition:"+ states[i] +"+STOP"
        transition_score = feature_dict.get(transition_key, -9999999)
        backward_scores[len(x)-1, i] = transition_score

    for i in range(len(x)-1,0,-1):
        for j  in range(len(states)):
            temp_score = 0 
            for k in range(len(states)):
                transition_key = "transition:" + states[j] + "+" + states[k]
                emission_key = "emission:"+ states[k] + "+" + x[i] 
                transition_score = feature_dict.get(transition_key, -9999999)
                emission_score = feature_dict.get(emission_key, -9999999)
                temp_score += np.exp(min(emission_score + transition_score + backward_scores[i, k], threshold))
            # Add to backward scores array
            
            
            if temp_score!=0:
                backward_scores[i-1, j] = math.log(temp_score)
            else:
                backward_scores[i-1, j] = -9999999
#             backward_scores[i-1, j] = math.log(temp_score) if temp_score else -9999999
    
    backward_prob = 0
    for i in range(len(states)):
        transition_key = "transition:" + "START" + "+" + states[i]
        emission_key = "emission:" + states[i] + "+" + x[0] # Emission of last word
        transition_score = feature_dict.get(transition_key, -9999999)
        emission_score = feature_dict.get(emission_key, -9999999)
        overall_score = math.exp(min(emission_score + transition_score + backward_scores[0, i], threshold))
        backward_prob += overall_score
        
    if backward_prob!=0:
        beta = math.log(backward_prob)
    else:
        beta = -threshold    
#     beta = math.log(backward_prob) if backward_prob else -9999999
    
    
    return backward_scores, beta

def forward_backward(x, feature_dict, states):
    
    threshold = 700
    
    forward_scores, alpha = compute_forward_score(x, feature_dict, states)
    forward_prob = math.exp(min(alpha, threshold))
    backward_scores, beta = compute_backward_score(x, feature_dict, states)
    backward_prob = math.exp(min(beta, threshold))
    feature_expected_counts = {} #defaultdict(float)

    for i in range(len(x)):
        for j in range(len(states)):
            emission_key = "emission:" + states[j] + "+" + x[i]
            feature_expected_counts[emission_key] = feature_expected_counts.get(emission_key, 0.0) + math.exp(min(forward_scores[i, j] + backward_scores[i, j] - alpha, threshold))
#             feature_expected_counts[emission_key] += np.exp(min(forward_scores[i, j] + backward_scores[i, j] - alpha, 700))
    
    for i in range(len(states)):
        start_transition_key =  "transition:" + "START" + "+" + states[i]
        feature_expected_counts[start_transition_key] = feature_expected_counts.get(start_transition_key, 0.0) + math.exp(min(forward_scores[0, i] + backward_scores[0, i] - alpha, threshold))
        stop_transition_key =  "transition:" + states[i] + "+"  + "STOP"
        feature_expected_counts[stop_transition_key] = feature_expected_counts.get(stop_transition_key, 0.0) + math.exp(min(forward_scores[len(x)-1, i] + backward_scores[len(x)-1, i] - alpha, threshold))
#         feature_expected_counts[stop_transition_key] += np.exp(min(forward_scores[len(x)-1, i] + backward_scores[len(x)-1, i] - alpha, 700))
    
    for i in range(len(states)):
        for j in range(len(states)):
            transition_key =  "transition:" + states[i] + "+"  + states[j]
            transition_score = feature_dict.get(transition_key, -9999999) 
            total = 0
            for k in range(len(x)-1):
                emission_key =  "emission:" + states[j] + "+"  + x[k+1]
                emission_score = feature_dict.get(emission_key, -9999999)

                total += np.exp(min(forward_scores[k, i] + backward_scores[k+1, j] + transition_score + emission_score - alpha, threshold))

            feature_expected_counts[transition_key] = total
    
    return feature_expected_counts


In [66]:
# This gives assertion error

# #YQ
def get_feature_count(x, y, feature_dict):
    # Input and output sequences must be of the same length
    #assert(len(x) == len(y))
    n = len(x) # Sequence length
    
    # Stores the number of times each feature appears in (x, y)
    feature_count = {}
    
    # Compute emission features
    for i in range(n):
        formatted_word = x[i]
        emission_key = "emission:"+y[i]+"+"+formatted_word
        feature_count[emission_key] = feature_count.get(emission_key, 0) + 1
    
    # Compute transition features
    # Add START and STOP tags to y
    updated_y = ["START"] + y + ["STOP"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = "transition:"+prev_y+"+"+y_i
        feature_count[transition_key] = feature_count.get(transition_key, 0) + 1
    
    return feature_count

In [67]:
#ANDRE
def compute_gradients(train_inputs, train_labels, f, states, nabla = 0, regularization = False):
    feature_gradients = {}
    for i in range(len(train_labels)):
        x = train_inputs[i]
        y = train_labels[i]
        feature_expected_counts = forward_backward(x, f, states)
        actual_counts = get_feature_count(x, y, f)
        
        for k, v in feature_expected_counts.items():
            feature_gradients[k] = feature_gradients.get(k, 0) + v
            
        for k, v in actual_counts.items():
            feature_gradients[k] = feature_gradients.get(k, 0) - v
    if regularization:
        for k, v in f.items():
            feature_gradients[k] += 2*nabla*f[k]

    return feature_gradients

In [68]:
states = list(set(data_tags))
train_inputs, train_labels = get_dataset(dataset + "/train")

# Comment the below section out for Spanish, as O+the doesn't exist!
# Check against numerical gradient for several values
feature_key_checks = ['emission:O+the', 'transition:START+O', 'transition:O+O']
feature_gradients = compute_gradients(train_inputs, train_labels, resulting_dict, states)
loss1 = compute_crf_loss(train_inputs, train_labels, resulting_dict, states)
delta = 1e-6

for feature_key in feature_key_checks:
    print("Checking", feature_key)
    new_f = resulting_dict.copy()
    new_f[feature_key] += delta

    loss2 = compute_crf_loss(train_inputs, train_labels, new_f, states)

    numerical_gradient = (loss2 - loss1) / delta
    analytical_gradient = feature_gradients[feature_key]
    
    # Ensure numerical and analytical gradient are close
    assert(abs(numerical_gradient - analytical_gradient) / max(abs(numerical_gradient), 1e-8) <= 1e-3)

Checking emission:O+the
Checking transition:START+O
Checking transition:O+O


In [69]:
loss2

465.45775458076787

In [70]:
numerical_gradient

-91.07647252903917

In [71]:
analytical_gradient

-91.07659201027221

In [72]:
# print(abs(numerical_gradient - analytical_gradient) / max(abs(numerical_gradient), 1e-8))

# Part 4


In [73]:
def numpy_to_dict(w, f, reverse = False):
    '''
    Converts a numpy array w to a dictionary with keys from f.
    '''
    for i,k in enumerate(f.keys()):
        f[k] = w[i]
    return f
def prepare_grad_for_bfgs(grads,f):
    '''
    Converts a dictionary to a numpy array.
    '''
    np_grads = np.zeros(len(f))
    for i,k in enumerate(f.keys()):
        np_grads[i] = grads[k]
    return np_grads

In [74]:
from scipy.optimize import fmin_l_bfgs_b 

states = list(set(data_tags))
train_inputs, train_labels = get_dataset(dataset + "/train")

def callbackF(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    '''
    loss = compute_crf_loss(train_inputs,train_labels,resulting_dict,states,0.1,regularization=True) 
    print('Loss:{0:.4f}'.format(loss))

def get_loss_grad(w, *args): 
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    Returns:
        loss: loss, float
        grads: gradients, numpy array
    '''

    train_inputs,train_labels,f,states = args
    f = numpy_to_dict(w, f)
    # compute loss and grad
    loss = compute_crf_loss(train_inputs, train_labels, f, states, 0.1, regularization=True)
    grads = compute_gradients(train_inputs, train_labels, f, states, 0.1, regularization=True)
    grads = prepare_grad_for_bfgs(grads, f) 
    # return loss and grad
    return loss, grads

init_w = np.zeros(len(resulting_dict))
result = fmin_l_bfgs_b(get_loss_grad, init_w, args=(train_inputs, train_labels, resulting_dict, states), pgtol=0.01, callback=callbackF)

Loss:945.7148
Loss:871.4124
Loss:798.5328
Loss:741.8041
Loss:666.4605
Loss:547.1342
Loss:468.2452
Loss:439.5731
Loss:424.3510
Loss:412.3042
Loss:391.1955
Loss:374.4352
Loss:366.4924
Loss:357.3852
Loss:351.6903
Loss:348.6322
Loss:344.8704
Loss:341.6333
Loss:340.3294
Loss:338.5670
Loss:337.7123
Loss:336.3834
Loss:336.0727
Loss:335.4178
Loss:334.6147
Loss:333.7576
Loss:333.4080
Loss:333.1507
Loss:332.9755
Loss:332.8540
Loss:332.6693
Loss:332.4352
Loss:332.2776
Loss:332.2402
Loss:332.1757
Loss:332.1033
Loss:332.0078
Loss:331.9674
Loss:331.9383
Loss:331.9215
Loss:331.9121
Loss:331.9083
Loss:331.8913
Loss:331.8865
Loss:331.8783
Loss:331.8698
Loss:331.8663
Loss:331.8642
Loss:331.8627
Loss:331.8605
Loss:331.8571
Loss:331.8562
Loss:331.8536
Loss:331.8529
Loss:331.8522
Loss:331.8515
Loss:331.8511
Loss:331.8505
Loss:331.8503
Loss:331.8500
Loss:331.8499
Loss:331.8496
Loss:331.8493
Loss:331.8492
Loss:331.8490
Loss:331.8489
Loss:331.8489


In [77]:
f = numpy_to_dict(result[0], resulting_dict)
decode(dataset+ "/dev.in", states, f, dataset+'/dev.p4.out')

g_tags, p_tags = eval(dataset+'/dev.p4.out',dataset+'/dev.out',dataset+'/eval4.out')
print(evaluate(g_tags,p_tags,verbose=True))

processed 2097 tokens with 236 phrases; found: 239 phrases; correct: 123.
accuracy:  53.82%; (non-O)
accuracy:  87.41%; precision:  51.46%; recall:  52.12%; FB1:  51.79
              art: precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              eve: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              geo: precision:  63.95%; recall:  64.71%; FB1:  64.33  86
              gpe: precision:  88.89%; recall:  64.00%; FB1:  74.42  18
              nat: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              org: precision:  40.00%; recall:  28.57%; FB1:  33.33  25
              per: precision:  44.00%; recall:  34.38%; FB1:  38.60  25
              tim: precision:  37.35%; recall:  58.49%; FB1:  45.59  83
(51.46443514644351, 52.118644067796616, 51.78947368421052)


# Part 6